In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33878
2,Huelva,Confirmados PDIA 14 días,858
3,Huelva,Tasa PDIA 14 días,"167,1960558879124"
4,Huelva,Confirmados PDIA 7 días,408
5,Huelva,Total Confirmados,34063
6,Huelva,Curados,32242
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33878.0
/tmp/ipykernel_65480/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9261.0
/tmp/ipykernel_65480/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_65480/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_65480/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_65480/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1514 personas en los últimos 7 días 

Un positivo PCR cada 631 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33878.0,408.0,858.0,513170.0,79.505817,167.196056,170.0
Huelva-Costa,19555.0,260.0,531.0,289548.0,89.795129,183.389283,102.0
Condado-Campiña,10996.0,119.0,270.0,156231.0,76.169262,172.821015,53.0
Huelva (capital),9261.0,95.0,228.0,143837.0,66.046984,158.512761,32.0
Isla Cristina,2246.0,49.0,67.0,21393.0,229.046884,313.186556,23.0
Sierra de Huelva-Andévalo Central,2968.0,24.0,51.0,67391.0,35.613064,75.677761,10.0
Almonte,1940.0,21.0,35.0,24507.0,85.689803,142.816338,10.0
Ayamonte,1387.0,24.0,53.0,21104.0,113.722517,251.137225,9.0
Gibraleón,670.0,14.0,21.0,12737.0,109.915993,164.873989,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,29.0,13.0,13.0,1043.0,1246.404602,1246.404602,8.0
Alosno,201.0,13.0,21.0,3933.0,330.536486,533.943555,5.0
Cerro de Andévalo (El),160.0,4.0,9.0,2327.0,171.895144,386.764074,3.0
Paymogo,79.0,1.0,4.0,1162.0,86.058520,344.234079,0.0
Bonares,313.0,9.0,20.0,6060.0,148.514851,330.033003,4.0
Isla Cristina,2246.0,49.0,67.0,21393.0,229.046884,313.186556,23.0
San Juan del Puerto,597.0,4.0,28.0,9411.0,42.503453,297.524174,2.0
Palos de la Frontera,739.0,7.0,32.0,11742.0,59.615057,272.525975,3.0
Ayamonte,1387.0,24.0,53.0,21104.0,113.722517,251.137225,9.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,579.0,1.0,8.0,7939.0,12.596045,100.768359,1.0,0.125000
San Juan del Puerto,597.0,4.0,28.0,9411.0,42.503453,297.524174,2.0,0.142857
Palos de la Frontera,739.0,7.0,32.0,11742.0,59.615057,272.525975,3.0,0.218750
Paymogo,79.0,1.0,4.0,1162.0,86.058520,344.234079,0.0,0.250000
Cartaya,1425.0,13.0,44.0,20083.0,64.731365,219.090773,4.0,0.295455
Aljaraque,1190.0,9.0,29.0,21474.0,41.911148,135.047034,7.0,0.310345
Rosal de la Frontera,34.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
Palma del Condado (La),1202.0,5.0,14.0,10801.0,46.292010,129.617628,0.0,0.357143
Moguer,1434.0,19.0,53.0,21867.0,86.888919,242.374354,3.0,0.358491
